In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import klib

In [ ]:
#load processed data
df = pd.read_csv('KDDTrain+_cleaned.csv')

In [12]:
X = df.drop(['type'], axis=1)
y = df['type']
# one hot encoding
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [13]:
# train a bi-lstm model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='bilstm_best_model.h5', verbose=1, save_best_only=True, monitor = 'val_accuracy', mode = 'max')
start = time.time()
# train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpointer])
end = time.time()
print(end - start)

Epoch 1/10
4209/4209 [==============================] - ETA: 0s - loss: 0.2101 - accuracy: 0.9226
Epoch 1: val_accuracy improved from -inf to 0.95964, saving model to bilstm_best_model.h5
4209/4209 [==============================] - 809s 191ms/step - loss: 0.2101 - accuracy: 0.9226 - val_loss: 0.1052 - val_accuracy: 0.9596
Epoch 2/10


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4209/4209 [==============================] - ETA: 0s - loss: 0.0753 - accuracy: 0.9720
Epoch 2: val_accuracy improved from 0.95964 to 0.98411, saving model to bilstm_best_model.h5
4209/4209 [==============================] - 840s 200ms/step - loss: 0.0753 - accuracy: 0.9720 - val_loss: 0.0438 - val_accuracy: 0.9841
Epoch 3/10
4209/4209 [==============================] - ETA: 0s - loss: 0.0527 - accuracy: 0.9809
Epoch 3: val_accuracy improved from 0.98411 to 0.98563, saving model to bilstm_best_model.h5
4209/4209 [==============================] - 909s 216ms/step - loss: 0.0527 - accuracy: 0.9809 - val_loss: 0.0374 - val_accuracy: 0.9856
Epoch 4/10
4209/4209 [==============================] - ETA: 0s - loss: 0.0428 - accuracy: 0.9840
Epoch 4: val_accuracy improved from 0.98563 to 0.98692, saving model to bilstm_best_model.h5
4209/4209 [==============================] - 847s 201ms/step - loss: 0.0428 - accuracy: 0.9840 - val_loss: 0.0414 - val_accuracy: 0.9869
Epoch 5/10
4209/4209 [=====

In [14]:
model = load_model('bilstm_best_model.h5')
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

2105/2105 [==============================] - 64s 30ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13340
           1       1.00      1.00      1.00     13507
           2       1.00      1.00      1.00     13499
           3       0.99      0.97      0.98     13629
           4       0.97      0.99      0.98     13368

    accuracy                           0.99     67343
   macro avg       0.99      0.99      0.99     67343
weighted avg       0.99      0.99      0.99     67343

0.9911052373669127


In [15]:
# from confusion matrix calculate accuracy of each class
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
print('dos:', cm[0,0])
print('normal:', cm[1,1])
print('probe:', cm[2,2])
print('r2l:', cm[3,3])
print('u2r:', cm[4,4])

Accuracy of each class:
dos: 0.9993253373313343
normal: 0.9997778929443992
probe: 0.9987406474553671
r2l: 0.9725585149313963
u2r: 0.9853381208856972
